In [7]:
import pandas as pd
from datetime import datetime

def dt(hour, minute, second=0):
    return datetime(2021, 1, 1, hour, minute, second)

data = [
    (None, None, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2, 0), dt(1, 2, 50)),
    (1, 1, dt(1, 2, 0), dt(2, 2, 1)),        
]

data = [
    (None, None, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2), dt(1, 10)),
    (1, 1, dt(1, 2, 0), dt(1, 2, 50)),
    (1, 1, dt(1, 2, 0), dt(2, 2, 1)),
    ]



columns = ['PUlocationID', 'DOlocationID', 'pickup_datetime', 'dropOff_datetime']
df = pd.DataFrame(data, columns=columns)

In [8]:
df.to_parquet("input_unit_test.parquet", engine='pyarrow', index=False)

In [9]:
df.head()

,PUlocationID,DOlocationID,pickup_datetime,dropOff_datetime
0,NaN,NaN,2021-01-01 01:02:00,2021-01-01 01:10:00
1,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:10:00
2,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:02:50
3,1.0,1.0,2021-01-01 01:02:00,2021-01-01 02:02:01


In [11]:
now_time= datetime.now()
month = now_time.month
year = now_time.year

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [12]:
df.head()

,PUlocationID,DOlocationID,pickup_datetime,dropOff_datetime,ride_id
0,NaN,NaN,2021-01-01 01:02:00,2021-01-01 01:10:00,2022/08_0
1,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:10:00,2022/08_1
2,1.0,1.0,2021-01-01 01:02:00,2021-01-01 01:02:50,2022/08_2
3,1.0,1.0,2021-01-01 01:02:00,2021-01-01 02:02:01,2022/08_3


In [20]:
#make expected output

import sys
import pickle
import pandas as pd
import datetime

with open('/home/ubuntu/mlops-zoomcamp_week6/06-best-practices/homework/model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)
    
    

categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [23]:
input_file='input_unit_test.parquet'
excepted_output="excepted_output.parquet"
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)


print('predicted mean duration:', y_pred.mean())


df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

df_result.to_parquet(excepted_output, engine='pyarrow', index=False)

predicted mean duration: 34.64434841620357


In [25]:
df_result.head()

,ride_id,predicted_duration
0,2022/08_0,23.052085
1,2022/08_1,46.236612


In [35]:
import test_batch

FileNotFoundError: [Errno 2] No such file or directory: 'model.bin'

In [31]:
pwd

'/home/ubuntu/mlops-zoomcamp_week6/06-best-practices/homework/tests'

In [32]:
!ls

__init__.py		 input_unit_test	  test_batch_utility.ipynb
__pycache__		 input_unit_test.parquet
excepted_output.parquet  test_batch.py


In [1]:
ls

__init__.py   excepted_output.parquet  test_batch.py
__pycache__/  input_unit_test.parquet  test_batch_utility.ipynb


In [2]:
cd ..


/home/ubuntu/mlops-zoomcamp_week6/06-best-practices/homework


In [3]:
ls


 Dockerfile     batch_intgration.py  'taxi_type=fhv_year=2021_month=02.parquet'
 Pipfile        docker-compose.yaml   test_batch.py
 Pipfile.lock   integration_test.py   test_ligic.ipynb
 __pycache__/   model.bin             tests/
 batch.py       run.sh*


In [8]:
!python batch.py 

predicted mean duration: 16.191691679979066
                 ride_id  predicted_duration
1              2021/02_1           14.539865
2              2021/02_2           13.740422
3              2021/02_3           15.593339
4              2021/02_4           15.188118
5              2021/02_5           13.817206
...                  ...                 ...
1037687  2021/02_1037687           12.433246
1037688  2021/02_1037688           12.910885
1037689  2021/02_1037689           15.834923
1037690  2021/02_1037690           16.783176
1037691  2021/02_1037691           19.654626

[990113 rows x 2 columns]
